# `tensorflow.keras` convnet from scratch

This notebook demonstrates the use of a simple convolutional network. On a 2-class classification task--that is, an ouput layer with two nodes activated via `softmax`. Accordingly, loss is calculated using the `categorical cross-entropy`.

References:

* The github repository from the BAGLS team contained in this [link](https://github.com/anki-xyz/bagls/blob/master/Utils/DataGenerator.py#L109)


In [1]:
# dev convenience
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("..")
import PATHS

import os
import numpy as np

os.environ["CUDA_VISIBLE_DEVICES"] = "0,2"
os.environ["WANDB_SILENT"] = "True"
os.environ["WANDB_NOTEBOOK_NAME"] = "03-simple-convnet.ipynb"

PROJECT_NAME = 'bagls-sh-test'
RUN_NAME = 'convnet_from_scratch'
METRICS_TABLE_NAME = 'metrics_table'
GRADCAM_LAYER_NAME = "conv2d_3"

In [3]:
import wandb
print("W&B: ", wandb.__version__)
wandb.login()

# # manage logs
# import logging

# logger = logging.getLogger("wandb")
# logger.setLevel(logging.ERROR)

# logging.getLogger('tensorflow').disabled = True

W&B:  0.13.5


True

In [4]:
# tf loader
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow import keras

In [5]:
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 5834541230411947289
]


2022-11-29 13:20:39.444974: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-29 13:20:39.452540: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-11-29 13:20:39.452599: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: jupyter-mdorosan
2022-11-29 13:20:39.452610: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: jupyter-mdorosan
2022-11-29 13:20:39.452731: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 515.65.1
2022-11-29 13:20:39.452762: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported v

In [7]:
import config
configs = config.nb_configs

In [8]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.imagenet_utils import preprocess_input

# initialize data generator
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    validation_split=configs["validation_split"],
    rescale=configs["rescale"],
    width_shift_range=configs["width_shift_range"],
    height_shift_range=configs["height_shift_range"],
    shear_range=configs["shear_range"],
    zoom_range=configs["zoom_range"],
    fill_mode=configs["fill_mode"],
    horizontal_flip=configs["horizontal_flip"],
    rotation_range=configs["rotation_range"],
)

test_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input, 
    rescale=configs["rescale"],
)

In [9]:
train_dir = configs["train_dir"]
test_dir = configs["test_dir"]

batch_size = configs["batch_size"]
class_names = configs["class_names"]
interpol = configs["interpol"]
cmap = configs["cmap"]
label_mode = configs["label_mode"]
labels = configs["labels"]
image_size = configs["image_size"]


train_dataset = train_datagen.flow_from_directory(
    directory=train_dir,
    target_size=image_size,
    color_mode=cmap,
    classes=class_names,
    class_mode=label_mode,
    batch_size=batch_size,
    interpolation=interpol,
    subset="training",
)

val_dataset = train_datagen.flow_from_directory(
    directory=train_dir,
    target_size=image_size,
    color_mode=cmap,
    classes=class_names,
    class_mode=label_mode,
    batch_size=batch_size,
    interpolation=interpol,
    subset="validation",
)

test_dataset = test_datagen.flow_from_directory(
    directory=test_dir,
    target_size=image_size,
    color_mode=cmap,
    classes=class_names,
    class_mode=label_mode,
    batch_size=batch_size,
    interpolation=interpol,
    shuffle=False, # do not shuffle for later evaluation, alphanum sort
)

configs.update({"val_steps": val_dataset.samples // configs["batch_size"]})

Found 526 images belonging to 2 classes.
Found 26 images belonging to 2 classes.
Found 33 images belonging to 2 classes.


In [10]:
from tensorflow import keras
from tensorflow.keras import layers

def define_model():
    inputs = keras.Input(shape=(*image_size, 3))
    x = layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu')(inputs)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)
    x = layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu')(x)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)
    x = layers.Conv2D(filters=128, kernel_size=(3, 3), activation='relu')(x)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)
    x = layers.Conv2D(filters=128, kernel_size=(3, 3), activation='relu')(x)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)
    x = layers.Flatten()(x)
    x = layers.Dense(units=512, activation='relu')(x)
    x = keras.layers.Dropout(configs['dropout_rate'])(x)
    outputs = layers.Dense(units=2, activation='softmax')(x)
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

In [11]:
tf.keras.backend.clear_session()
model = define_model()

2022-11-28 16:21:12.052050: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10419 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:04:00.0, compute capability: 6.1


In [12]:
from tensorflow.keras import metrics
thresh = configs["thresh"]
metrics_dict = {
    "ACC":  metrics.BinaryAccuracy(name="ACC", threshold=thresh),
    "AUC-ROC": metrics.AUC(name='ROC', curve='ROC'),
    "AUC-PR": metrics.AUC(name='PR', curve='PR'),
    "TP": metrics.TruePositives(name="TP", thresholds=thresh),
    "TN": metrics.TrueNegatives(name="TN", thresholds=thresh),
    "FP": metrics.FalsePositives(name="FP", thresholds=thresh),
    "FN": metrics.FalseNegatives(name="FN", thresholds=thresh),
}

In [13]:
from tensorflow.keras import optimizers
from tensorflow.keras import losses

# opt = optimizers.Adam(learning_rate=1e-06)
opt = optimizers.Adam()
met = list(metrics_dict.values())

model.compile(
    loss=losses.CategoricalCrossentropy(),
    optimizer=opt,
    metrics=met,
)

In [14]:
# verify arch
# base predictions with untrained classif head
base_preds = model.predict(test_dataset)
base_preds

2022-11-28 16:21:13.161400: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-11-28 16:21:14.525161: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8400


array([[0.4947047 , 0.50529534],
       [0.4928894 , 0.5071106 ],
       [0.49209926, 0.5079007 ],
       [0.5010365 , 0.49896345],
       [0.49397966, 0.50602037],
       [0.49059653, 0.50940347],
       [0.4948586 , 0.50514144],
       [0.4933839 , 0.5066161 ],
       [0.49469635, 0.5053037 ],
       [0.4935302 , 0.50646985],
       [0.49260017, 0.50739986],
       [0.49810368, 0.5018963 ],
       [0.48776394, 0.51223606],
       [0.49051067, 0.5094893 ],
       [0.49354976, 0.5064502 ],
       [0.4975986 , 0.5024014 ],
       [0.49231964, 0.50768036],
       [0.49403486, 0.5059651 ],
       [0.49445748, 0.5055425 ],
       [0.49248135, 0.50751865],
       [0.49917343, 0.5008266 ],
       [0.4907841 , 0.50921583],
       [0.4952608 , 0.5047392 ],
       [0.49545157, 0.50454843],
       [0.4994172 , 0.5005828 ],
       [0.48993823, 0.5100618 ],
       [0.48913926, 0.51086074],
       [0.49550295, 0.5044971 ],
       [0.48889464, 0.51110536],
       [0.48411545, 0.5158846 ],
       [0.

In [16]:
from interpretation import ValLog, GRADCamLogger

# initialize run
run = wandb.init(
    project=PROJECT_NAME, 
    name=RUN_NAME,
    config=configs, 
    job_type='train',
)

wandb_callback = wandb.keras.WandbCallback(
    monitor="val_ROC",
    mode="max",
    save_model=True,
    save_graph=True,
    compute_flops=True,
)

callbacks = [
    wandb_callback,
    ValLog(generator=val_dataset, num_log_batches=1),
    GRADCamLogger(generator=test_dataset, layer_name=GRADCAM_LAYER_NAME, num_log_batches=1)
    
]

wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


In [17]:
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=configs["epochs"], 
    shuffle=True,
    callbacks=callbacks,
)
run.finish()

2022-11-28 16:21:25.660987: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 1
2022-11-28 16:21:25.661240: I tensorflow/core/grappler/clusters/single_machine.cc:357] Starting new session
2022-11-28 16:21:25.664270: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10419 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:04:00.0, compute capability: 6.1
2022-11-28 16:21:25.667943: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:1137] Optimization results for grappler item: graph_to_optimize
  function_optimizer: function_optimizer did nothing. time = 0.012ms.
  function_optimizer: function_optimizer did nothing. time = 0.002ms.



Instructions for updating:
Use `tf.compat.v1.graph_util.tensor_shape_from_node_def_name`
Epoch 1/5
9/9 [==============================] - 13s 1s/step - loss: 0.7178 - ACC: 0.5722 - ROC: 0.5932 - PR: 0.5772 - TP: 301.0000 - TN: 301.0000 - FP: 225.0000 - FN: 225.0000 - val_loss: 0.6560 - val_ACC: 0.6154 - val_ROC: 0.6879 - val_PR: 0.6571 - val_TP: 16.0000 - val_TN: 16.0000 - val_FP: 10.0000 - val_FN: 10.0000


2022-11-28 16:21:41.382516: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: /home/mdorosan/2022/bagls-sh-project/notebooks/wandb/run-20221128_162115-1kz9kxz7/files/model-best/assets


wandb: Adding directory to artifact (/home/mdorosan/2022/bagls-sh-project/notebooks/wandb/run-20221128_162115-1kz9kxz7/files/model-best)... Done. 0.7s


...

Epoch 3/5
9/9 [==============================] - 9s 1s/step - loss: 0.6490 - ACC: 0.6502 - ROC: 0.6847 - PR: 0.6559 - TP: 342.0000 - TN: 342.0000 - FP: 184.0000 - FN: 184.0000 - val_loss: 0.5819 - val_ACC: 0.7692 - val_ROC: 0.7885 - val_PR: 0.7204 - val_TP: 20.0000 - val_TN: 20.0000 - val_FP: 6.0000 - val_FN: 6.0000
INFO:tensorflow:Assets written to: /home/mdorosan/2022/bagls-sh-project/notebooks/wandb/run-20221128_162115-1kz9kxz7/files/model-best/assets


wandb: Adding directory to artifact (/home/mdorosan/2022/bagls-sh-project/notebooks/wandb/run-20221128_162115-1kz9kxz7/files/model-best)... Done. 0.7s


Epoch 4/5
9/9 [==============================] - 9s 1s/step - loss: 0.6278 - ACC: 0.6597 - ROC: 0.7138 - PR: 0.6872 - TP: 347.0000 - TN: 347.0000 - FP: 179.0000 - FN: 179.0000 - val_loss: 0.5621 - val_ACC: 0.7308 - val_ROC: 0.7855 - val_PR: 0.7812 - val_TP: 19.0000 - val_TN: 19.0000 - val_FP: 7.0000 - val_FN: 7.0000
Epoch 5/5
9/9 [==============================] - 10s 1s/step - loss: 0.6357 - ACC: 0.6597 - ROC: 0.6975 - PR: 0.6655 - TP: 347.0000 - TN: 347.0000 - FP: 179.0000 - FN: 179.0000 - val_loss: 0.6027 - val_ACC: 0.7308 - val_ROC: 0.8003 - val_PR: 0.8015 - val_TP: 19.0000 - val_TN: 19.0000 - val_FP: 7.0000 - val_FN: 7.0000
INFO:tensorflow:Assets written to: /home/mdorosan/2022/bagls-sh-project/notebooks/wandb/run-20221128_162115-1kz9kxz7/files/model-best/assets


wandb: Adding directory to artifact (/home/mdorosan/2022/bagls-sh-project/notebooks/wandb/run-20221128_162115-1kz9kxz7/files/model-best)... Done. 0.7s


ACC,▁▅▇██
FN,█▅▂▁▁
FP,█▅▂▁▁
PR,▁▃▆█▇
ROC,▁▄▆█▇
TN,▁▅▇██
TP,▁▅▇██
epoch,▁▃▅▆█
loss,█▄▃▁▂
val_ACC,▁██▆▆
val_FN,█▁▁▃▃


In [18]:
run = wandb.init(project=PROJECT_NAME, job_type="inference", name=RUN_NAME)
model_at = run.use_artifact("model-" + RUN_NAME + ":latest")
model_dir = model_at.download()
print("model: ", model_dir)
best_model = keras.models.load_model(model_dir)

wandb: Downloading large artifact model-convnet_from_scratch:latest, 111.03MB. 4 files... 
wandb:   4 of 4 files downloaded.  
Done. 0:0:0.3


model:  ./artifacts/model-convnet_from_scratch:v3


In [19]:
# test metrics
import utils

metrics_results = best_model.evaluate(test_dataset)
metrics_results = dict(zip(["loss"] + list(metrics_dict.keys()), 
                           metrics_results))
tp, fp, tn, fn = (metrics_results["TP"], metrics_results["FP"], 
                  metrics_results["TN"], metrics_results["FN"])

add_metrics = {
    "SENSITIVITY": utils.get_sensitivity(tp, fp, tn, fn),
    "SPECIFICTY": utils.get_specificity(tp, fp, tn, fn),
    "PPV": utils.get_ppv(tp, fp, tn, fn),
    "NPV": utils.get_npv(tp, fp, tn, fn),
    "F1" : utils.get_fbeta(tp, fp, tn, fn, beta=1),
}
metrics_results.update(add_metrics)

print(f"Metrics: \n", metrics_results)

columns = list(metrics_results.keys())
metrics_table = wandb.Table(columns=columns)
metrics_table.add_data(*metrics_results.values())
wandb.run.log({METRICS_TABLE_NAME : metrics_table})

# add logging of confusion matrix image from matplotlib
           
# get preds
trained_preds = best_model.predict(test_dataset)
run.finish()

1/1 [==============================] - 1s 1s/step - loss: 0.6329 - ACC: 0.5455 - ROC: 0.7199 - PR: 0.7676 - TP: 18.0000 - TN: 18.0000 - FP: 15.0000 - FN: 15.0000
Metrics: 
 {'loss': 0.6329288482666016, 'ACC': 0.5454545617103577, 'AUC-ROC': 0.7199265956878662, 'AUC-PR': 0.7675637006759644, 'TP': 18.0, 'TN': 18.0, 'FP': 15.0, 'FN': 15.0, 'SENSITIVITY': 0.5454545454545454, 'SPECIFICTY': 0.5454545454545454, 'PPV': 0.5454545454545454, 'NPV': 0.5454545454545454, 'F1': 0.5454545454545454}


## End